In [1]:
import os
import requests
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
from time import sleep
from bs4 import BeautifulSoup
import re
from ipywidgets import IntProgress
from IPython.display import display

In [2]:
# Spotify API Keys
spotify_cliend_id='bca78196e824433fbdf88ec18d84825f'
spotify_client_secret='d43763215bd8435eb9b3faaf048ca038'
os.environ['SPOTIPY_CLIENT_ID']=spotify_cliend_id
os.environ['SPOTIPY_CLIENT_SECRET']=spotify_client_secret

In [3]:
# Generate API Token
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
def get_features(title, artist, year=None):
    year_part=f"year:{year}" if year is not None else ''
    artist = str(artist)
    if ',' in artist:
        artists = artist.split(',')
        for artist in artists:
            features = get_features(title, artist)
            if features is not None:
                return features
            
    results = sp.search(q=f'track:{title} artist:{artist} {year_part}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        results = sp.search(q=f'{title} artist:{artist} {year_part}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        results = sp.search(q=f'{title} {year_part}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        results = sp.search(q=f'artist:{artist} {year_part}', type='track', limit=1)
    if len(results['tracks']['items']) == 0:
        return
    track_info = results['tracks']['items'][0]
    track_id = track_info['id']
    features = sp.audio_features([track_id])[0]
    
    if features is None:
        return
    
    features_as_dict = {
        'URL': track_info['external_urls']['spotify'],
        'TrackId': track_id,
        'Acousticness': features['acousticness'],
        'Danceability': features['danceability'],
        'Duration (ms)': features['duration_ms'],
        'Energy': features['energy'],
        'Instrumentalness': features['instrumentalness'],
        'Key': features['key'],
        'Liveness': features['liveness'],
        'Loudness': features['loudness'],
        'Mode': features['mode'],
        'Speechiness': features['speechiness'],
        'Tempo': features['tempo'],
        'Time Signature': features['time_signature'],
        'Valence': features['valence'],
        'Album': track_info['album']['name'],
        'Image': track_info['album']['images'][0]['url'] if len(track_info['album']['images'])>0 else None,
        'Explicit': track_info['explicit'],
        'Popularity': track_info['popularity'],
        'Preview': track_info['preview_url']
    }
    return features_as_dict

In [ ]:
get_features('Roadhouse Blues', 'Doors')

{'URL': 'https://open.spotify.com/track/1Q5kgpp4pmyGqPwNBzkSrw',
 'TrackId': '1Q5kgpp4pmyGqPwNBzkSrw',
 'Acousticness': 0.337,
 'Danceability': 0.612,
 'Duration (ms)': 243827,
 'Energy': 0.634,
 'Instrumentalness': 4.6e-05,
 'Key': 9,
 'Liveness': 0.147,
 'Loudness': -8.212,
 'Mode': 1,
 'Speechiness': 0.0319,
 'Tempo': 121.059,
 'Time Signature': 4,
 'Valence': 0.904,
 'Album': 'Morrison Hotel',
 'Image': 'https://i.scdn.co/image/34b1c4afdd8e576fb048e9e6c900c6c9fe33ea76',
 'Explicit': False,
 'Popularity': 71,
 'Preview': 'https://p.scdn.co/mp3-preview/ce4e9952e1519b9fe6c858b085fbe79077ec9dfb?cid=bca78196e824433fbdf88ec18d84825f'}

In [ ]:
get_features('Mame', 'Angela Lansbury, Bea Arthur, Jane Connell, Charles Braswell, Jerry Lanning, Frankie Michaels')

{'URL': 'https://open.spotify.com/track/4nmD7mHFACe4XwcoZ7wWsy',
 'TrackId': '4nmD7mHFACe4XwcoZ7wWsy',
 'Acousticness': 0.536,
 'Danceability': 0.355,
 'Duration (ms)': 344853,
 'Energy': 0.828,
 'Instrumentalness': 5.6e-05,
 'Key': 2,
 'Liveness': 0.931,
 'Loudness': -10.261,
 'Mode': 0,
 'Speechiness': 0.487,
 'Tempo': 136.013,
 'Time Signature': 4,
 'Valence': 0.238,
 'Album': 'Tap Your Troubles Away! - The Words and Music of Jerry Herman',
 'Image': 'https://i.scdn.co/image/4b4b77ce331bbbbc7d696516649ad758d632f313',
 'Explicit': False,
 'Popularity': 3,
 'Preview': 'https://p.scdn.co/mp3-preview/ca80ddfa159bc9ec96b615e6c106d1e2efb8978f?cid=bca78196e824433fbdf88ec18d84825f'}

In [ ]:
get_features('Ella Swings Lightly', 'Ella Fitzgerald', 1958)

{'URL': 'https://open.spotify.com/track/3ZxPZcLbINzbetZFcJthvU',
 'TrackId': '3ZxPZcLbINzbetZFcJthvU',
 'Acousticness': 0.838,
 'Danceability': 0.627,
 'Duration (ms)': 156467,
 'Energy': 0.131,
 'Instrumentalness': 0,
 'Key': 10,
 'Liveness': 0.223,
 'Loudness': -13.779,
 'Mode': 1,
 'Speechiness': 0.0503,
 'Tempo': 143.087,
 'Time Signature': 4,
 'Valence': 0.351,
 'Album': 'Ella Swings Lightly',
 'Image': 'https://i.scdn.co/image/7f33383709040fba7b408770758b9b0d7ba4239a',
 'Explicit': False,
 'Popularity': 13,
 'Preview': None}

In [ ]:
get_features('Ella Fitzgerald Sings The Duke Ellington Song Book', 'Ella Fitzgerald', 1958)

{'URL': 'https://open.spotify.com/track/5FY0EikZVSBOwpjQa9S5Ii',
 'TrackId': '5FY0EikZVSBOwpjQa9S5Ii',
 'Acousticness': 0.675,
 'Danceability': 0.615,
 'Duration (ms)': 138320,
 'Energy': 0.186,
 'Instrumentalness': 0,
 'Key': 0,
 'Liveness': 0.194,
 'Loudness': -12.382,
 'Mode': 0,
 'Speechiness': 0.0508,
 'Tempo': 73.007,
 'Time Signature': 4,
 'Valence': 0.749,
 'Album': 'Ella Fitzgerald Sings The Irving Berlin Song Book',
 'Image': 'https://i.scdn.co/image/3350581fb4712a44a6f6b50e234f6539c291b425',
 'Explicit': False,
 'Popularity': 34,
 'Preview': None}

In [ ]:
get_features('Hev Yew Gotta Loight, Boy?', 'The Singing Postman', year=1964)

In [ ]:
GRAMMY_WINNERS = os.path.join('..', 'data', 'grammy-winners.csv')
GRAMMY_WINNERS_ENRICHED = os.path.join('..', 'data', 'grammy-winners-enriched.csv')
SONGS_RELEASED = os.path.join('..', 'data', 'songs-released.csv')
SONGS_RELEASED_ENRICHED = os.path.join('..', 'data', 'songs-released-enriched.csv')
TOP_100_SONGS = os.path.join('..', 'data', 'top-100-songs.csv')
TOP_100_SONGS_ENRICHED = os.path.join('..', 'data', 'top-100-songs-enriched.csv')

In [ ]:
grammy_winners_df = pd.read_csv(GRAMMY_WINNERS)
grammy_winners_df.rename(columns={'annualGrammy':'Year', 'awardFor':'Title', 'name':'Artist'}, inplace=True)
grammy_winners_df.drop(labels=['awardType', 'category'], axis=1, inplace=True)
grammy_winners_df['Year'] = grammy_winners_df.Year + 1957
grammy_winners_df.drop_duplicates(inplace=True)
grammy_winners_df = grammy_winners_df[['Year', 'Artist', 'Title']]
grammy_winners_df.head()

,Year,Artist,Title
0,1958,Domenico Modugno,Nel Blu Dipinto Di Blu (Volare)
1,1958,Henry Mancini,The Music From Peter Gunn
2,1958,"Domenico Modugno, songwriter (Domenico Modugno)",Nel Blu Dipinto Di Blu (Volare)
3,1958,Ella Fitzgerald,Ella Fitzgerald Sings The Irving Berlin Song Book
4,1958,Perry Como,Catch A Falling Star


In [ ]:
songs_released_df = pd.read_csv(SONGS_RELEASED)
songs_released_df.drop_duplicates(inplace=True)
songs_released_df = songs_released_df[['Year', 'Artist', 'Title']]
songs_released_df.head()

,Year,Artist,Title
0,1958,Clyde McPhatter,A Lover's Question
1,1958,Pat Boone,A Wonderful Time Up There
2,1958,The Everly Brothers,All I Have to Do Is Dream
3,1958,Andy Williams,Are You Sincere
4,1958,The Playmates,Beep Beep


In [ ]:
top_100_songs_df = pd.read_csv(TOP_100_SONGS)
top_100_songs_df.drop_duplicates(inplace=True)
top_100_songs_df = top_100_songs_df[['Year', 'Artist', 'Title']]
top_100_songs_df.drop_duplicates(inplace=True)
top_100_songs_df.head()

,Year,Artist,Title
0,1958,Domenico Modugno,Volare
1,1958,Everly Brothers,All I Have To Do Is Dream / Claudette
2,1958,Elvis Presley,Don't / I Beg of You
3,1958,David Seville,Witch Doctor
4,1958,Pérez Prado,Patricia


In [ ]:
import time
def enrich_with_features(df):
    pb = IntProgress(min=0, max=len(df))
    display(pb)
    failures = []
    for index, row in df.iterrows():
        time.sleep(4)
        features = get_features(row['Title'], row['Artist'], row['Year'])
        pb.value += 1
        if features is None:
            failures.append((row['Title'], row['Artist'], row['Year']))
            continue
        for k, v in features.items():
            df.loc[index, k] = v
            
    print(f'Number of failures - {len(failures)}')
    print(failures)
    return df

In [ ]:
grammy_winners_df.shape

(3911, 3)

In [ ]:
grammy_winners_enriched_df = enrich_with_features(grammy_winners_df)
grammy_winners_enriched_df.head()

IntProgress(value=0, max=3911)

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [ ]:
grammy_winners_enriched_df.shape

In [ ]:
grammy_winners_enriched_df.to_csv(GRAMMY_WINNERS_ENRICHED, index=False)

In [ ]:
songs_released_df.shape

In [ ]:
songs_released_enriched_df = enrich_with_features(songs_released_df)
songs_released_enriched_df.head()

In [ ]:
songs_released_enriched_df.shape

In [ ]:
songs_released_enriched_df.to_csv(SONGS_RELEASED_ENRICHED, index=False)

In [ ]:
top_100_songs_df.shape

In [ ]:
top_100_songs_enriched_df = enrich_with_features(top_100_songs_df)
top_100_songs_enriched_df.head()

In [ ]:
top_100_songs_enriched_df.shape

In [ ]:
top_100_songs_enriched_df.to_csv(TOP_100_SONGS_ENRICHED, index=False)